SVM using BUG dataset

In [98]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from lime import lime_text
from lime.lime_text import LimeTextExplainer
import pickle

In [99]:
# load data
dataset = pd.read_csv('datasets/BUG/balanced_BUG.csv')
train, test = train_test_split(dataset, test_size=0.2, random_state=42)
# get samples with only neutral or stereotype sentence
train = train[train['stereotype'].isin([0, 1])]
test = test[test['stereotype'].isin([0, 1])]

vectorizer = CountVectorizer()
trainTexts = vectorizer.fit_transform(train['sentence_text'])
testTexts = vectorizer.transform(test['sentence_text'])

xTrain = pd.DataFrame(trainTexts.toarray(), columns=vectorizer.get_feature_names_out())
xTest = pd.DataFrame(testTexts.toarray(), columns=vectorizer.get_feature_names_out())
yTrain = train['predicted gender']
yTest = test['predicted gender']

# train model
model = LinearSVC()
model.fit(xTrain, yTrain)

model = CalibratedClassifierCV(model, method='sigmoid', cv='prefit')
model.fit(xTrain, yTrain)

with open('svmModel.pkl', 'wb') as model_file:
    pickle.dump((model, vectorizer), model_file)

/Users/byroncuachin/anaconda3/lib/python3.11/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


In [100]:
# accuracy
pred = model.predict(xTest)

accuracy = accuracy_score(yTest, pred)
print("Accuracy:", accuracy)

# confusion matrix
cm = confusion_matrix(yTest, pred)
print("Confusion matrix:\n", cm)

Accuracy: 0.9875097580015613
Confusion matrix:
 [[1280   15]
 [  17 1250]]


In [101]:
text = [
"""
Once upon a time in the vibrant city of Harmonyville , there lived a college student named Mia Rodriguez . Mia was a junior majoring in environmental science at Rivertide University . Her love for nature and a deep sense of responsibility towards the planet fueled her determination to make a positive impact . One sunny afternoon , Mia stumbled upon a notice about the annual Green Innovation Challenge —an event where students could pitch eco -friendly projects to a panel of environmental experts . Inspired and eager to contribute , Mia decided to develop a sustainable urban gardening initiative called  "GreenHaven . " With her hands in the soil and a heart full of passion , Mia transformed an unused corner of the campus into a thriving community garden . She envisioned GreenHaven as a place where students could come together , learn about sustainable agriculture , and cultivate their own fruits and vegetables . Mia believed that this initiative could not only promote environmental consciousness but also foster a sense of community among her peers . As the garden flourished , so did Mia 's connection with her fellow students . The project became a hub of creativity , where ideas for sustainable living blossomed alongside the vibrant array of fruits and vegetables . Mia 's dedication and leadership drew the attention of both students and faculty alike . When the day of the Green Innovation Challenge arrived , Mia nervously but proudly presented GreenHaven to the panel of judges . The vision , dedication , and positive impact of her project resonated deeply , earning her the first prize and a scholarship for further environmental studies . Word of Mia 's success spread , and GreenHaven became a symbol of sustainable living on campus . Mia 's journey didn 't end with the competition ; instead , it marked the beginning of a new chapter . With the scholarship in hand , Mia continued her studies , conducting research on innovative ways to create sustainable urban environments . As Mia graduated from Rivertide University , she left behind a legacy of green initiatives and a campus that had been transformed by the power of community and sustainability . GreenHaven continued to thrive , inspiring future generations of students to think creatively about environmental issues . Mia 's story became a beacon of hope , showing that even a single college student with a passion for change could make a lasting impact on the world . And so , as Mia embarked on her journey beyond college , she carried with her not just a degree but the knowledge that small , meaningful actions could ripple into waves of positive transformation for the planet and its people .
""",
"""
Once upon a time in the vibrant city of Harmonyville ,  lived a college student named   .  was a junior majoring in environmental science at Rivertide University .  love for nature and a deep sense of responsibility towards the planet fueled  determination to make a positive impact . One sunny afternoon ,  stumbled upon a notice about the annual Green Innovation Challenge — an event where students could pitch eco -friendly projects to a panel of environmental experts . Inspired and eager to contribute ,  decided to develop a sustainable urban gardening initiative called   " GreenHaven . " With  hands in the soil and a heart full of passion ,  transformed an unused corner of the campus into a thriving community garden .  envisioned GreenHaven as a place where students could come together , learn about sustainable agriculture , and cultivate  own fruits and vegetables .  believed that this initiative could not only promote environmental consciousness but also foster a sense of community among  peers . As the garden flourished , so did  's connection with  fellow students . The project became a hub of creativity , where ideas for sustainable living blossomed alongside the vibrant array of fruits and vegetables .  's dedication and leadership drew the attention of both students and faculty alike . When the day of the Green Innovation Challenge arrived ,  nervously but proudly presented GreenHaven to the panel of judges . The vision , dedication , and positive impact of  project resonated deeply , earning  the first prize and a scholarship for further environmental studies . Word of  's success spread , and  became a symbol of sustainable living on campus .  's    t end with the competition ; instead ,  marked the beginning of a new chapter . With the scholarship in hand ,  continued  studies , conducting research on innovative ways to create sustainable urban environments . As  graduated from Rivertide University ,  left behind a legacy of green initiatives and a campus  had been transformed by the power of community and sustainability . GreenHaven continued to thrive , inspiring future generations of students to think creatively about environmental issues .  's story became a beacon of hope , showing that even a single college student with a passion for change could make a lasting impact on the world . And so , as  embarked on  journey beyond college ,  carried with  not just a degree but the knowledge  small , meaningful actions could ripple into waves of positive transformation for the planet and  people .
""",
"""
Once upon a time in the bustling city of Arcadia , there lived a college student named Alex Reynolds . Alex was a junior majoring in computer science at the prestigious Arcadia University . He was a diligent student with a passion for coding and a penchant for exploring the world of technology . One day , as Alex was immersed in his studies at the campus library , he stumbled upon an intriguing flyer .
""",
"""
Once upon a time in the bustling city of Arcadia , there lived a college student . This student was a junior majoring in computer science at the prestigious Arcadia University . They were a diligent student with a passion for coding and a penchant for exploring the world of technology . One day , as this student was immersed in their studies at the campus library , they stumbled upon an intriguing flyer .
""",
"""
Once upon a time in the vibrant college town of Crestwood , there lived a spirited young student named Emily . She was a sophomore at Crestwood University , pursuing a degree in environmental science with dreams of making a positive impact on the planet . Emily was known for her boundless enthusiasm , infectious energy , and a love for exploring the world around her . One crisp autumn day , Emily stumbled upon a flyer for an environmental awareness club called  "Green Harmony " on the college bulletin board . Intrigued and passionate about environmental causes , she decided to attend their next meeting . As Emily walked into the meeting room , she was greeted by a diverse group of students who shared her passion for sustainability . The club was planning an ambitious project to transform an unused campus space into a thriving community garden . Emily was immediately captivated by the idea and eagerly joined the efforts . The days turned into weeks as the Green Harmony team worked tirelessly , planning , planting , and nurturing their garden . Emily found herself forming deep connections with her fellow club members as they faced challenges and celebrated victories together . The project not only brought life to the neglected space but also breathed new life into Emily 's college experience . Amidst the busy academic schedule , Emily discovered a sense of purpose beyond textbooks and exams . The garden project became a symbol of unity , showcasing the power of collaboration and the positive impact a group of dedicated individuals could make . As the garden flourished , so did Emily 's personal growth . She learned about sustainable practices , organic gardening , and the importance of community engagement . Her once mundane college routine transformed into a fulfilling journey , where every day brought new lessons and opportunities . Word of the Green Harmony garden spread throughout the campus , attracting attention from both students and faculty . The college recognized the club 's efforts and even awarded them a grant to expand their project . Emily and her friends found themselves at the forefront of a campus -wide movement toward sustainability . Through this journey , Emily discovered not only her passion for environmental science but also her ability to lead and inspire change . The once ordinary college student had become a beacon of hope and inspiration for those around her . As the seasons changed , so did Emily and her friends , leaving behind a legacy of a thriving community garden and a more environmentally conscious campus . The tale of the college student who discovered her purpose in the embrace of nature and community echoed through the years , inspiring future generations of students to dream big and make a difference in the world .
""",
"""
Once upon a time in the vibrant college town of Crestwood ,  lived a spirited young student named  .  was a sophomore at Crestwood University , pursuing a degree in environmental science with dreams of making a positive impact on the planet . Emily was known for  boundless enthusiasm , infectious energy , and a love for exploring the world around  . One crisp autumn day , Emily stumbled upon a flyer for an environmental awareness club called   " Green Harmony " on the college bulletin board . Intrigued and passionate about environmental causes ,  decided to attend  next meeting . As Emily walked into the meeting room ,  was greeted by a diverse group of students  shared  passion for sustainability . The club was planning an ambitious project to transform an unused campus space into a thriving community garden . Emily was immediately captivated by the idea and eagerly joined the efforts . The days turned into weeks as the   team worked tirelessly , planning , planting , and nurturing  garden . Emily found  forming deep connections with  fellow club members as  faced challenges and celebrated victories together . The project not only brought life to the neglected space but also breathed new life into Emily 's college experience . Amidst the busy academic schedule , Emily discovered a sense of purpose beyond textbooks and exams . The garden project became a symbol of unity , showcasing the power of collaboration and the positive impact a group of dedicated individuals could make . As the garden flourished , so did  's personal growth .  learned about sustainable practices , organic gardening , and the importance of community engagement .  once mundane college routine transformed into a fulfilling journey , where every day brought new lessons and opportunities . Word of the Green Harmony garden spread throughout the campus , attracting attention from both students and faculty . The college recognized the club 's efforts and even awarded  a grant to expand  project . Emily and  friends found  at the forefront of a campus -wide movement toward sustainability . Through this journey , Emily discovered not only  passion for environmental science but also  ability to lead and inspire change . The once ordinary college student had become a beacon of hope and inspiration for  around  . As the seasons changed , so did  and  friends , leaving behind a legacy of a thriving community garden and a more environmentally conscious campus . The tale of the college student  discovered  purpose in the embrace of nature and community echoed through the years , inspiring future generations of students to dream big and make a difference in the world .
""",
"""
In the vibrant city of Rivertown , there lived a college student named Jason Harris . Jason was a senior at Rivertown University , majoring in journalism and aspiring to be a storyteller . He had a keen interest in human experiences and a passion for shedding light on untold stories . One day , while perusing the local newspaper , Jason stumbled upon an article about an elderly woman named Mrs . Eleanor Bennett , who had spent decades working as a librarian in the city . Intrigued by the snippet of her life story , Jason felt compelled to dig deeper and share her tale with the world . After some research , Jason learned that Mrs . Bennett had not only been a dedicated librarian but had also been involved in community initiatives , fostering a love for reading among children and organizing events to bring people together . However , her own life story had largely remained in the shadows . Determined to uncover the full narrative , Jason reached out to Mrs . Bennett and asked for an interview . To his surprise , she welcomed the opportunity to share her story , and soon they sat down for a conversation that would unveil a lifetime of experiences . As Jason delved into Mrs . Bennett 's past , he discovered a woman who had overcome personal challenges , witnessed historical events , and touched the lives of many in Rivertown . Her journey was filled with moments of joy , heartbreak , and resilience , making for a compelling narrative that Jason couldn 't wait to share with his readers . With meticulous attention to detail , Jason crafted a feature article that not only highlighted Mrs . Bennett 's contributions to the community but also captured the essence of her character . The story resonated with readers , and the local newspaper received an overwhelming response . Mrs . Bennett became a beloved figure in Rivertown , celebrated for her dedication and the richness of her life story . The success of the article opened doors for Jason , who found himself on the path to becoming a respected journalist . Inspired by Mrs . Bennett 's story , he continued to seek out and share the untold narratives within his community , becoming a storyteller who bridged the gaps between generations . As graduation day approached , Jason reflected on his college journey . His time at Rivertown University had not only provided him with an education but had also instilled in him the power of storytelling to connect people and inspire change . With a heart full of gratitude and a portfolio filled with impactful stories , Jason ventured into the world , ready to make a difference , one narrative at a time .
""",
"""
In the vibrant city of Rivertown ,  lived a college student named   .  was a senior at Rivertown University , majoring in journalism and aspiring to be a storyteller .  had a keen interest in human experiences and a passion for shedding light on untold stories . One day , while perusing the local newspaper ,  stumbled upon an article about an elderly woman named  .   ,  had spent decades working as a librarian in the city . Intrigued by the snippet of  life story ,  felt compelled to dig deeper and share  tale with the world . After some research ,  learned that  .  had not only been a dedicated librarian but had also been involved in community initiatives , fostering a love for reading among children and organizing events to bring people together . However ,  own life story had largely remained in the shadows . Determined to uncover the full narrative ,  reached out to  .  and asked for an interview . To  surprise ,  welcomed the opportunity to share  story , and soon  sat down for a conversation  would unveil a lifetime of experiences . As  delved into  .  's past ,  discovered a woman  had overcome personal challenges , witnessed historical events , and touched the lives of many in Rivertown .  journey was filled with moments of joy , heartbreak , and resilience , making for a compelling narrative   couldn ' t wait to share with  readers . With meticulous attention to detail ,  crafted a feature article that not only highlighted  .  's contributions to the community but also captured the essence of  character . The story resonated with readers , and the local newspaper received an overwhelming response . Mrs .  became a beloved figure in Rivertown , celebrated for  dedication and the richness of  life story . The success of the article opened doors for  ,  found  on the path to becoming a respected journalist . Inspired by  .  's story ,  continued to seek out and share the untold narratives within  community , becoming a storyteller  bridged the gaps between generations . As graduation day approached ,  reflected on  college journey .  time at Rivertown University had not only provided  with an education but had also instilled in  the power of storytelling to connect people and inspire change . With a heart full of gratitude and a portfolio filled with impactful stories ,  ventured into the world , ready to make a difference , one narrative at a time .
""",
"""
In the bustling town of Summitville , there lived a college man named Lucas Turner . Lucas was a mechanical engineering major at Summitville Tech , known for his passion for building things from the ground up . His college life was a dynamic mix of late -night design sessions , hands -on projects , and a camaraderie with his fellow engineering students . One day , Lucas stumbled upon an old , abandoned workshop on the outskirts of town . The sight of forgotten tools and discarded materials sparked a fire in him . Inspired by the potential of the neglected space , he decided to breathe new life into it and create a collaborative hub for engineering students . Lucas rallied a group of like -minded friends , and together they transformed the workshop into the  "Innovation Forge . " It became a haven for students to work on personal projects , exchange ideas , and bring their engineering dreams to life . The space buzzed with the sounds of drills , the clanking of metal , and the hum of 3D printers as students collaborated on everything from robotics to sustainable energy solutions . As the Innovation Forge gained popularity , Lucas and his team decided to organize an annual engineering expo , showcasing the innovative projects that emerged from their workshop . The expo not only attracted students and faculty but also local businesses and industry professionals keen on discovering emerging talent . One year , Lucas and his team developed a prototype for a solar -powered irrigation system designed to help local farmers in Summitville . The project gained attention not only for its engineering prowess but also for its potential to make a real impact in the community . The Innovation Forge became a focal point for collaborative projects that aimed to address real -world challenges . Lucas 's journey through college was not just about earning a degree ; it was about leaving a lasting mark on the engineering community . As graduation approached , the Innovation Forge continued to thrive , and Lucas felt a sense of pride in the legacy he had created . The town of Summitville now had a vibrant hub of innovation , thanks to the hard work and dedication of Lucas Turner . His college experience became a testament to the transformative power of hands -on learning , collaboration , and the drive to make a positive impact on the world through engineering ingenuity .
""",
"""
In the bustling town of Summitville ,  lived a college man named   . Lucas was a mechanical engineering major at Summitville Tech , known for  passion for building things from the ground up .  college life was a dynamic mix of late -night design sessions , hands -on projects , and a camaraderie with  fellow engineering students . One day , Lucas stumbled upon an old , abandoned workshop on the outskirts of town . The sight of forgotten tools and discarded materials sparked a fire in  . Inspired by the potential of the neglected space ,  decided to breathe new life into  and create a collaborative hub for engineering students . Lucas rallied a group of like -minded friends , and together  transformed the workshop into the   " Innovation Forge . "  became a haven for students to work on personal projects , exchange ideas , and bring  engineering dreams to life . The space buzzed with the sounds of drills , the clanking of metal , and the hum of 3D printers as students collaborated on  from robotics to sustainable energy solutions . As the Innovation Forge gained popularity , Lucas and  team decided to organize an annual engineering expo , showcasing the innovative projects  emerged from  workshop . The expo not only attracted students and faculty but also local businesses and industry professionals keen on discovering emerging talent . One year , Lucas and  team developed a prototype for a solar -powered irrigation system designed to help local farmers in Summitville . The project gained attention not only for  engineering prowess but also for  potential to make a real impact in the community . The Innovation Forge became a focal point for collaborative projects  aimed to address real -world challenges . Lucas 's journey through college was not just about earning a degree ;  was about leaving a lasting mark on the engineering community . As graduation approached , the Innovation Forge continued to thrive , and Lucas felt a sense of pride in the legacy  had created . The town of Summitville now had a vibrant hub of innovation , thanks to the hard work and dedication of Lucas Turner .  college experience became a testament to the transformative power of hands -on learning , collaboration , and the drive to make a positive impact on the world through engineering ingenuity .
"""
]
vectorizedText = vectorizer.transform(text)
textsTransformed = pd.DataFrame(vectorizedText.toarray(), columns=vectorizer.get_feature_names_out())
predProb = model.predict_proba(textsTransformed)
pred = model.predict(textsTransformed)

yActual = [0, 0, 1, 1, 0, 0, 1, 1, 1, 1]
explainer = LimeTextExplainer(class_names=['female', 'male'])
for i in range(len(pred)):
    print(i + 1, ":")
    print("Male: ", predProb[i][1])
    print("Female: ", predProb[i][0])
    print("Predicted: ", pred[i])
    if(yActual[i] == 0):
        print("Actual: Female")
    else:
        print("Actual: Male")
    
    vectorized_text = vectorizer.transform([text[i]])
    predict_function = lambda x: model.predict_proba(vectorizer.transform(x))
    explanation = explainer.explain_instance(text[i], predict_function, num_features=20)
    top_words_lime = explanation.as_list()
    print(f"Top words for text response {i + 1}:")
    masculineWords = []
    feminineWords = []
    for word, score in top_words_lime:
        if score > 0:
            masculineWords.append((word, score))
        else:
            feminineWords.append((word, score))
    print("Masculine words: ", masculineWords)
    print("Feminine words: ", feminineWords)
    print("")





1 :
Male:  4.583407051718981e-37
Female:  1.0
Predicted:  female
Actual: Female


/Users/byroncuachin/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but LinearSVC was fitted with feature names
  warnings.warn(


Top words for text response 1:
Masculine words:  [('of', 0.023889305706260525), ('the', 0.022509045058220045), ('Once', 0.012033914260813664), ('to', 0.010223661226169098), ('first', 0.008815718908405752), ('actions', 0.008113367510618861), ('come', 0.007773714505959946), ('also', 0.0077422289356570495), ('arrived', 0.00645465036596769), ('vibrant', 0.0058471199630072196)]
Feminine words:  [('her', -0.05492489415291262), ('she', -0.052026689164497576), ('student', -0.05097987049085918), ('She', -0.037901470544518145), ('Her', -0.03451036896138427), ('as', -0.018732021455593953), ('became', -0.014145100571237263), ('green', -0.013132546944907071), ('about', -0.013024794635806221), ('competition', -0.011167348553064737)]

2 :
Male:  3.34660035449224e-05
Female:  0.9999665339964551
Predicted:  female
Actual: Female


/Users/byroncuachin/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but LinearSVC was fitted with feature names
  warnings.warn(


Top words for text response 2:
Masculine words:  [('of', 0.14362471047594233), ('the', 0.1357491075917767), ('come', 0.05058507783604051), ('to', 0.03861815568236246), ('Once', 0.03732183201363074), ('college', 0.03711929049188478), ('for', 0.034373453134801744), ('first', 0.034106625552241525), ('on', 0.03249697400115549), ('but', 0.026373538774423156)]
Feminine words:  [('student', -0.5287127311015), ('about', -0.10404458133331347), ('could', -0.07925132555997587), ('Green', -0.07154953000089609), ('became', -0.06837650012458879), ('living', -0.061800364457321386), ('where', -0.060177757079863836), ('even', -0.05806464750505931), ('When', -0.056182151291778906), ('green', -0.05033345471613056)]

3 :
Male:  0.702690851134862
Female:  0.29730914886513804
Predicted:  male
Actual: Male
Top words for text response 3:
Masculine words:  [('his', 0.27493393457741755), ('He', 0.23502540984000567), ('he', 0.22072718571959624), ('Once', 0.07789895472747387), ('the', 0.0759986763273506), ('Unive

...

/Users/byroncuachin/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but LinearSVC was fitted with feature names
  warnings.warn(


Top words for text response 8:
Masculine words:  [('the', 0.14332065308514733), ('had', 0.10798278989004066), ('life', 0.10535782596940162), ('of', 0.07793515587329246), ('for', 0.06420278739259198), ('to', 0.06254268516770548), ('an', 0.05939795710955695), ('full', 0.053904532785219075), ('local', 0.044427309966408784)]
Feminine words:  [('student', -0.23273687703500479), ('librarian', -0.22902344390291418), ('journalist', -0.17803176832153317), ('story', -0.14051504220780553), ('in', -0.10437711890953182), ('Mrs', -0.10147196146953831), ('out', -0.08183499298341494), ('people', -0.0660600863251217), ('own', -0.0651661754604483), ('only', -0.06166529445491519), ('readers', -0.0562966459310846)]

9 :
Male:  1.0
Female:  0.0
Predicted:  male
Actual: Male


/Users/byroncuachin/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but LinearSVC was fitted with feature names
  warnings.warn(


Top words for text response 9:
Masculine words:  [('his', 0.03457657349794363), ('His', 0.03191807819576094), ('he', 0.029450326309635507), ('the', 0.025282155797568782), ('life', 0.020660279256804432), ('of', 0.015879389698596896), ('to', 0.014156954212146283), ('college', 0.012535517750240279), ('attention', 0.01115760667814439), ('for', 0.007734224694972404)]
Feminine words:  [('night', -0.010496947147545514), ('sessions', -0.010421998588474764), ('was', -0.010228453059071423), ('major', -0.010013182788536888), ('like', -0.009571037098178031), ('work', -0.008976747454607652), ('him', -0.008266036721240556), ('only', -0.0079735973549353), ('help', -0.007866937425115132), ('became', -0.007709632513977954)]

10 :
Male:  0.9999548140259166
Female:  4.518597408342817e-05
Predicted:  male
Actual: Male
Top words for text response 10:
Masculine words:  [('the', 0.13073391738000564), ('of', 0.09611817325470597), ('life', 0.09594084362702518), ('on', 0.0720740251843928), ('to', 0.061611019079

/Users/byroncuachin/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but LinearSVC was fitted with feature names
  warnings.warn(


Random Forest Using BUG Dataset

In [105]:
# load data
dataset = pd.read_csv('datasets/BUG/balanced_BUG.csv')
train, test = train_test_split(dataset, test_size=0.2, random_state=42)
# get samples with only neutral or stereotype sentence
train = train[train['stereotype'].isin([0, 1])]
test = test[test['stereotype'].isin([0, 1])]

vectorizer = CountVectorizer()
trainTexts = vectorizer.fit_transform(train['sentence_text'])
testTexts = vectorizer.transform(test['sentence_text'])

xTrain = pd.DataFrame(trainTexts.toarray(), columns=vectorizer.get_feature_names_out())
xTest = pd.DataFrame(testTexts.toarray(), columns=vectorizer.get_feature_names_out())
yTrain = train['predicted gender']
yTest = test['predicted gender']

# train model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(xTrain, yTrain)

with open('randomForestModel.pkl', 'wb') as model_file:
    pickle.dump((model, vectorizer), model_file)

In [106]:
# accuracy
pred = model.predict(xTest)

accuracy = accuracy_score(yTest, pred)
print("Accuracy:", accuracy)

# confusion matrix
cm = confusion_matrix(yTest, pred)
print("Confusion matrix:\n", cm)

Accuracy: 0.9914129586260734
Confusion matrix:
 [[1291    4]
 [  18 1249]]


In [107]:
text = [
"""
Once upon a time in the vibrant city of Harmonyville , there lived a college student named Mia Rodriguez . Mia was a junior majoring in environmental science at Rivertide University . Her love for nature and a deep sense of responsibility towards the planet fueled her determination to make a positive impact . One sunny afternoon , Mia stumbled upon a notice about the annual Green Innovation Challenge —an event where students could pitch eco -friendly projects to a panel of environmental experts . Inspired and eager to contribute , Mia decided to develop a sustainable urban gardening initiative called  "GreenHaven . " With her hands in the soil and a heart full of passion , Mia transformed an unused corner of the campus into a thriving community garden . She envisioned GreenHaven as a place where students could come together , learn about sustainable agriculture , and cultivate their own fruits and vegetables . Mia believed that this initiative could not only promote environmental consciousness but also foster a sense of community among her peers . As the garden flourished , so did Mia 's connection with her fellow students . The project became a hub of creativity , where ideas for sustainable living blossomed alongside the vibrant array of fruits and vegetables . Mia 's dedication and leadership drew the attention of both students and faculty alike . When the day of the Green Innovation Challenge arrived , Mia nervously but proudly presented GreenHaven to the panel of judges . The vision , dedication , and positive impact of her project resonated deeply , earning her the first prize and a scholarship for further environmental studies . Word of Mia 's success spread , and GreenHaven became a symbol of sustainable living on campus . Mia 's journey didn 't end with the competition ; instead , it marked the beginning of a new chapter . With the scholarship in hand , Mia continued her studies , conducting research on innovative ways to create sustainable urban environments . As Mia graduated from Rivertide University , she left behind a legacy of green initiatives and a campus that had been transformed by the power of community and sustainability . GreenHaven continued to thrive , inspiring future generations of students to think creatively about environmental issues . Mia 's story became a beacon of hope , showing that even a single college student with a passion for change could make a lasting impact on the world . And so , as Mia embarked on her journey beyond college , she carried with her not just a degree but the knowledge that small , meaningful actions could ripple into waves of positive transformation for the planet and its people .
""",
"""
Once upon a time in the vibrant city of Harmonyville ,  lived a college student named   .  was a junior majoring in environmental science at Rivertide University .  love for nature and a deep sense of responsibility towards the planet fueled  determination to make a positive impact . One sunny afternoon ,  stumbled upon a notice about the annual Green Innovation Challenge — an event where students could pitch eco -friendly projects to a panel of environmental experts . Inspired and eager to contribute ,  decided to develop a sustainable urban gardening initiative called   " GreenHaven . " With  hands in the soil and a heart full of passion ,  transformed an unused corner of the campus into a thriving community garden .  envisioned GreenHaven as a place where students could come together , learn about sustainable agriculture , and cultivate  own fruits and vegetables .  believed that this initiative could not only promote environmental consciousness but also foster a sense of community among  peers . As the garden flourished , so did  's connection with  fellow students . The project became a hub of creativity , where ideas for sustainable living blossomed alongside the vibrant array of fruits and vegetables .  's dedication and leadership drew the attention of both students and faculty alike . When the day of the Green Innovation Challenge arrived ,  nervously but proudly presented GreenHaven to the panel of judges . The vision , dedication , and positive impact of  project resonated deeply , earning  the first prize and a scholarship for further environmental studies . Word of  's success spread , and  became a symbol of sustainable living on campus .  's    t end with the competition ; instead ,  marked the beginning of a new chapter . With the scholarship in hand ,  continued  studies , conducting research on innovative ways to create sustainable urban environments . As  graduated from Rivertide University ,  left behind a legacy of green initiatives and a campus  had been transformed by the power of community and sustainability . GreenHaven continued to thrive , inspiring future generations of students to think creatively about environmental issues .  's story became a beacon of hope , showing that even a single college student with a passion for change could make a lasting impact on the world . And so , as  embarked on  journey beyond college ,  carried with  not just a degree but the knowledge  small , meaningful actions could ripple into waves of positive transformation for the planet and  people .
""",
"""
Once upon a time in the bustling city of Arcadia , there lived a college student named Alex Reynolds . Alex was a junior majoring in computer science at the prestigious Arcadia University . He was a diligent student with a passion for coding and a penchant for exploring the world of technology . One day , as Alex was immersed in his studies at the campus library , he stumbled upon an intriguing flyer .
""",
"""
Once upon a time in the bustling city of Arcadia , there lived a college student . This student was a junior majoring in computer science at the prestigious Arcadia University . They were a diligent student with a passion for coding and a penchant for exploring the world of technology . One day , as this student was immersed in their studies at the campus library , they stumbled upon an intriguing flyer .
""",
"""
Once upon a time in the vibrant college town of Crestwood , there lived a spirited young student named Emily . She was a sophomore at Crestwood University , pursuing a degree in environmental science with dreams of making a positive impact on the planet . Emily was known for her boundless enthusiasm , infectious energy , and a love for exploring the world around her . One crisp autumn day , Emily stumbled upon a flyer for an environmental awareness club called  "Green Harmony " on the college bulletin board . Intrigued and passionate about environmental causes , she decided to attend their next meeting . As Emily walked into the meeting room , she was greeted by a diverse group of students who shared her passion for sustainability . The club was planning an ambitious project to transform an unused campus space into a thriving community garden . Emily was immediately captivated by the idea and eagerly joined the efforts . The days turned into weeks as the Green Harmony team worked tirelessly , planning , planting , and nurturing their garden . Emily found herself forming deep connections with her fellow club members as they faced challenges and celebrated victories together . The project not only brought life to the neglected space but also breathed new life into Emily 's college experience . Amidst the busy academic schedule , Emily discovered a sense of purpose beyond textbooks and exams . The garden project became a symbol of unity , showcasing the power of collaboration and the positive impact a group of dedicated individuals could make . As the garden flourished , so did Emily 's personal growth . She learned about sustainable practices , organic gardening , and the importance of community engagement . Her once mundane college routine transformed into a fulfilling journey , where every day brought new lessons and opportunities . Word of the Green Harmony garden spread throughout the campus , attracting attention from both students and faculty . The college recognized the club 's efforts and even awarded them a grant to expand their project . Emily and her friends found themselves at the forefront of a campus -wide movement toward sustainability . Through this journey , Emily discovered not only her passion for environmental science but also her ability to lead and inspire change . The once ordinary college student had become a beacon of hope and inspiration for those around her . As the seasons changed , so did Emily and her friends , leaving behind a legacy of a thriving community garden and a more environmentally conscious campus . The tale of the college student who discovered her purpose in the embrace of nature and community echoed through the years , inspiring future generations of students to dream big and make a difference in the world .
""",
"""
Once upon a time in the vibrant college town of Crestwood ,  lived a spirited young student named  .  was a sophomore at Crestwood University , pursuing a degree in environmental science with dreams of making a positive impact on the planet . Emily was known for  boundless enthusiasm , infectious energy , and a love for exploring the world around  . One crisp autumn day , Emily stumbled upon a flyer for an environmental awareness club called   " Green Harmony " on the college bulletin board . Intrigued and passionate about environmental causes ,  decided to attend  next meeting . As Emily walked into the meeting room ,  was greeted by a diverse group of students  shared  passion for sustainability . The club was planning an ambitious project to transform an unused campus space into a thriving community garden . Emily was immediately captivated by the idea and eagerly joined the efforts . The days turned into weeks as the   team worked tirelessly , planning , planting , and nurturing  garden . Emily found  forming deep connections with  fellow club members as  faced challenges and celebrated victories together . The project not only brought life to the neglected space but also breathed new life into Emily 's college experience . Amidst the busy academic schedule , Emily discovered a sense of purpose beyond textbooks and exams . The garden project became a symbol of unity , showcasing the power of collaboration and the positive impact a group of dedicated individuals could make . As the garden flourished , so did  's personal growth .  learned about sustainable practices , organic gardening , and the importance of community engagement .  once mundane college routine transformed into a fulfilling journey , where every day brought new lessons and opportunities . Word of the Green Harmony garden spread throughout the campus , attracting attention from both students and faculty . The college recognized the club 's efforts and even awarded  a grant to expand  project . Emily and  friends found  at the forefront of a campus -wide movement toward sustainability . Through this journey , Emily discovered not only  passion for environmental science but also  ability to lead and inspire change . The once ordinary college student had become a beacon of hope and inspiration for  around  . As the seasons changed , so did  and  friends , leaving behind a legacy of a thriving community garden and a more environmentally conscious campus . The tale of the college student  discovered  purpose in the embrace of nature and community echoed through the years , inspiring future generations of students to dream big and make a difference in the world .
""",
"""
In the vibrant city of Rivertown , there lived a college student named Jason Harris . Jason was a senior at Rivertown University , majoring in journalism and aspiring to be a storyteller . He had a keen interest in human experiences and a passion for shedding light on untold stories . One day , while perusing the local newspaper , Jason stumbled upon an article about an elderly woman named Mrs . Eleanor Bennett , who had spent decades working as a librarian in the city . Intrigued by the snippet of her life story , Jason felt compelled to dig deeper and share her tale with the world . After some research , Jason learned that Mrs . Bennett had not only been a dedicated librarian but had also been involved in community initiatives , fostering a love for reading among children and organizing events to bring people together . However , her own life story had largely remained in the shadows . Determined to uncover the full narrative , Jason reached out to Mrs . Bennett and asked for an interview . To his surprise , she welcomed the opportunity to share her story , and soon they sat down for a conversation that would unveil a lifetime of experiences . As Jason delved into Mrs . Bennett 's past , he discovered a woman who had overcome personal challenges , witnessed historical events , and touched the lives of many in Rivertown . Her journey was filled with moments of joy , heartbreak , and resilience , making for a compelling narrative that Jason couldn 't wait to share with his readers . With meticulous attention to detail , Jason crafted a feature article that not only highlighted Mrs . Bennett 's contributions to the community but also captured the essence of her character . The story resonated with readers , and the local newspaper received an overwhelming response . Mrs . Bennett became a beloved figure in Rivertown , celebrated for her dedication and the richness of her life story . The success of the article opened doors for Jason , who found himself on the path to becoming a respected journalist . Inspired by Mrs . Bennett 's story , he continued to seek out and share the untold narratives within his community , becoming a storyteller who bridged the gaps between generations . As graduation day approached , Jason reflected on his college journey . His time at Rivertown University had not only provided him with an education but had also instilled in him the power of storytelling to connect people and inspire change . With a heart full of gratitude and a portfolio filled with impactful stories , Jason ventured into the world , ready to make a difference , one narrative at a time .
""",
"""
In the vibrant city of Rivertown ,  lived a college student named   .  was a senior at Rivertown University , majoring in journalism and aspiring to be a storyteller .  had a keen interest in human experiences and a passion for shedding light on untold stories . One day , while perusing the local newspaper ,  stumbled upon an article about an elderly woman named  .   ,  had spent decades working as a librarian in the city . Intrigued by the snippet of  life story ,  felt compelled to dig deeper and share  tale with the world . After some research ,  learned that  .  had not only been a dedicated librarian but had also been involved in community initiatives , fostering a love for reading among children and organizing events to bring people together . However ,  own life story had largely remained in the shadows . Determined to uncover the full narrative ,  reached out to  .  and asked for an interview . To  surprise ,  welcomed the opportunity to share  story , and soon  sat down for a conversation  would unveil a lifetime of experiences . As  delved into  .  's past ,  discovered a woman  had overcome personal challenges , witnessed historical events , and touched the lives of many in Rivertown .  journey was filled with moments of joy , heartbreak , and resilience , making for a compelling narrative   couldn ' t wait to share with  readers . With meticulous attention to detail ,  crafted a feature article that not only highlighted  .  's contributions to the community but also captured the essence of  character . The story resonated with readers , and the local newspaper received an overwhelming response . Mrs .  became a beloved figure in Rivertown , celebrated for  dedication and the richness of  life story . The success of the article opened doors for  ,  found  on the path to becoming a respected journalist . Inspired by  .  's story ,  continued to seek out and share the untold narratives within  community , becoming a storyteller  bridged the gaps between generations . As graduation day approached ,  reflected on  college journey .  time at Rivertown University had not only provided  with an education but had also instilled in  the power of storytelling to connect people and inspire change . With a heart full of gratitude and a portfolio filled with impactful stories ,  ventured into the world , ready to make a difference , one narrative at a time .
""",
"""
In the bustling town of Summitville , there lived a college man named Lucas Turner . Lucas was a mechanical engineering major at Summitville Tech , known for his passion for building things from the ground up . His college life was a dynamic mix of late -night design sessions , hands -on projects , and a camaraderie with his fellow engineering students . One day , Lucas stumbled upon an old , abandoned workshop on the outskirts of town . The sight of forgotten tools and discarded materials sparked a fire in him . Inspired by the potential of the neglected space , he decided to breathe new life into it and create a collaborative hub for engineering students . Lucas rallied a group of like -minded friends , and together they transformed the workshop into the  "Innovation Forge . " It became a haven for students to work on personal projects , exchange ideas , and bring their engineering dreams to life . The space buzzed with the sounds of drills , the clanking of metal , and the hum of 3D printers as students collaborated on everything from robotics to sustainable energy solutions . As the Innovation Forge gained popularity , Lucas and his team decided to organize an annual engineering expo , showcasing the innovative projects that emerged from their workshop . The expo not only attracted students and faculty but also local businesses and industry professionals keen on discovering emerging talent . One year , Lucas and his team developed a prototype for a solar -powered irrigation system designed to help local farmers in Summitville . The project gained attention not only for its engineering prowess but also for its potential to make a real impact in the community . The Innovation Forge became a focal point for collaborative projects that aimed to address real -world challenges . Lucas 's journey through college was not just about earning a degree ; it was about leaving a lasting mark on the engineering community . As graduation approached , the Innovation Forge continued to thrive , and Lucas felt a sense of pride in the legacy he had created . The town of Summitville now had a vibrant hub of innovation , thanks to the hard work and dedication of Lucas Turner . His college experience became a testament to the transformative power of hands -on learning , collaboration , and the drive to make a positive impact on the world through engineering ingenuity .
""",
"""
In the bustling town of Summitville ,  lived a college man named   . Lucas was a mechanical engineering major at Summitville Tech , known for  passion for building things from the ground up .  college life was a dynamic mix of late -night design sessions , hands -on projects , and a camaraderie with  fellow engineering students . One day , Lucas stumbled upon an old , abandoned workshop on the outskirts of town . The sight of forgotten tools and discarded materials sparked a fire in  . Inspired by the potential of the neglected space ,  decided to breathe new life into  and create a collaborative hub for engineering students . Lucas rallied a group of like -minded friends , and together  transformed the workshop into the   " Innovation Forge . "  became a haven for students to work on personal projects , exchange ideas , and bring  engineering dreams to life . The space buzzed with the sounds of drills , the clanking of metal , and the hum of 3D printers as students collaborated on  from robotics to sustainable energy solutions . As the Innovation Forge gained popularity , Lucas and  team decided to organize an annual engineering expo , showcasing the innovative projects  emerged from  workshop . The expo not only attracted students and faculty but also local businesses and industry professionals keen on discovering emerging talent . One year , Lucas and  team developed a prototype for a solar -powered irrigation system designed to help local farmers in Summitville . The project gained attention not only for  engineering prowess but also for  potential to make a real impact in the community . The Innovation Forge became a focal point for collaborative projects  aimed to address real -world challenges . Lucas 's journey through college was not just about earning a degree ;  was about leaving a lasting mark on the engineering community . As graduation approached , the Innovation Forge continued to thrive , and Lucas felt a sense of pride in the legacy  had created . The town of Summitville now had a vibrant hub of innovation , thanks to the hard work and dedication of Lucas Turner .  college experience became a testament to the transformative power of hands -on learning , collaboration , and the drive to make a positive impact on the world through engineering ingenuity .
"""
]
vectorizedText = vectorizer.transform(text)
textsTransformed = pd.DataFrame(vectorizedText.toarray(), columns=vectorizer.get_feature_names_out())
predProb = model.predict_proba(textsTransformed)
pred = model.predict(textsTransformed)

yActual = [0, 0, 1, 1, 0, 0, 1, 1, 1, 1]
explainer = LimeTextExplainer(class_names=['female', 'male'])
for i in range(len(pred)):
    print(i + 1, ":")
    print("Male: ", predProb[i][1])
    print("Female: ", predProb[i][0])
    print("Predicted: ", pred[i])
    if(yActual[i] == 0):
        print("Actual: Female")
    else:
        print("Actual: Male")
        
    vectorized_text = vectorizer.transform([text[i]])
    predict_function = lambda x: model.predict_proba(vectorizer.transform(x))
    explanation = explainer.explain_instance(text[i], predict_function, num_features=20)
    top_words_lime = explanation.as_list()
    print(f"Top words for text response {i + 1}:")
    masculineWords = []
    feminineWords = []
    for word, score in top_words_lime:
        if score > 0:
            masculineWords.append((word, score))
        else:
            feminineWords.append((word, score))
    print("Masculine words: ", masculineWords)
    print("Feminine words: ", feminineWords)
    print("")





1 :
Male:  0.3
Female:  0.7
Predicted:  female
Actual: Female


/Users/byroncuachin/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


Top words for text response 1:
Masculine words:  [('an', 0.03881001779009798), ('continued', 0.026671373748517576), ('the', 0.02019839133436416), ('about', 0.01995379882675494), ('to', 0.017195105329217388), ('peers', 0.017055799526091528), ('on', 0.01666617960909363), ('of', 0.015776486540651814), ('among', 0.015042518271359991), ('believed', 0.014206935528434984), ('their', 0.013368955561480775), ('arrived', 0.01334768551170358)]
Feminine words:  [('student', -0.04484451583553814), ('Her', -0.039660207745517366), ('her', -0.03957481434568894), ('She', -0.022581092633221236), ('she', -0.02250244533506502), ('students', -0.01950212957354425), ('story', -0.005017598065172543), ('only', -0.003234681191705741)]

2 :
Male:  0.41
Female:  0.59
Predicted:  female
Actual: Female


/Users/byroncuachin/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


Top words for text response 2:
Masculine words:  [('an', 0.029857424437760568), ('the', 0.023363149421702745), ('believed', 0.01859610148463281), ('about', 0.017640703755345814), ('in', 0.01626962691520522), ('of', 0.012417207577566444), ('peers', 0.012148653703924534), ('arrived', 0.011232703740408355), ('full', 0.010473605070417634), ('new', 0.010298294240615534)]
Feminine words:  [('student', -0.08204084598683085), ('students', -0.05787155048408309), ('that', -0.017731685743295145), ('story', -0.012221270216606031), ('only', -0.012078621339060182), ('ideas', -0.008947841347620361), ('community', -0.008744167391669184), ('had', -0.007755757885560713), ('presented', -0.006837024159138517), ('studies', -0.004367784493341503)]

3 :
Male:  0.67
Female:  0.33
Predicted:  male
Actual: Male


...

/Users/byroncuachin/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


Top words for text response 8:
Masculine words:  [('about', 0.01841146361468445), ('children', 0.017909506833597762), ('After', 0.01650359869710947), ('also', 0.013089135134471606), ('of', 0.012988669262363254), ('life', 0.01191638281418575), ('into', 0.011066444443682022), ('remained', 0.011056957809876287), ('local', 0.01038374200715613), ('at', 0.010080801233865221)]
Feminine words:  [('journalist', -0.053045085915520526), ('student', -0.03621574635354447), ('Mrs', -0.02668528311169106), ('librarian', -0.02118522551014899), ('experiences', -0.02080142322363682), ('article', -0.013629075664216894), ('character', -0.01180944338940139), ('woman', -0.010593216742426024), ('stories', -0.009635764633306348), ('human', -0.008396341372931661)]

9 :
Male:  0.66
Female:  0.34
Predicted:  male
Actual: Male


/Users/byroncuachin/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


Top words for text response 9:
Masculine words:  [('his', 0.043109753920806716), ('His', 0.039365378451328516), ('he', 0.03853802484147003), ('One', 0.01840432036098858), ('team', 0.013365499962925374), ('of', 0.011916978450077897), ('him', 0.01174460927252988), ('personal', 0.011111901798917495), ('design', 0.008633511628620945), ('their', 0.008422795876121485), ('through', 0.007449658805397833), ('decided', 0.007076573744367237)]
Feminine words:  [('students', -0.062345079653131234), ('experience', -0.052441328124246206), ('had', -0.01668762847557768), ('sight', -0.013618244900212713), ('old', -0.013347925969784156), ('ideas', -0.012973942822968475), ('talent', -0.012918478056609489), ('night', -0.010981438051236166)]

10 :
Male:  0.55
Female:  0.45
Predicted:  male
Actual: Male
Top words for text response 10:
Masculine words:  [('design', 0.0262314305752681), ('of', 0.026096467033496516), ('building', 0.01862708940634001), ('about', 0.015616077845728979), ('was', 0.01517807391232122

/Users/byroncuachin/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
